<a href="https://colab.research.google.com/github/edgardominguez23/Project-TI-1/blob/main/TI_1_Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final: Encuesta Nacional sobre Uso del Tiempo (ENUT) 2019

**Nombre:** Edgar Alan Dominguez Murillo  
**e-mail:** edgar.dominguez@alumnos.udg.mx

**Nombre:** Abraham de Jesus Cisneros Jarquin   
**e-mail:** abraham.cisneros6728@alumnos.udg.mx

## Resumen
<p align="justify">
La Encuesta Nacional sobre Uso del Tiempo (ENUT) 2019 tiene como objetivo proporcionar información estadística para la medición de todas las formas de trabajo de los individuos, tanto remunerado como no remunerado. Esta encuesta busca hacer visible la importancia de la producción doméstica y su contribución a la economía, así como la forma como usan su tiempo los hombres y las mujeres, los niños y las niñas, las personas de diferentes grupos de edad y las personas de diferentes niveles socioeconómicos.

## Objetivo
<p align="justify">
El objetivo de este proyecto analizar de manera detallada cuantos hombres y mujeres realizan tareas escolares, se analizan el total de personas que trabajan y hacen tareas escolares, cuantos no trabajan y realizan tareas escolares y ademas se toma una muestra de hombres y mujeres menores a 35 años que realizan tareas, se toma este rango de edad ya que es el periodo de la vida que comprender al rango escolar, entonces se busca analizar cuanta gente de la encuestada, aun asiste a la escuela, la informacion se obtiene utilizando la información recopilada por la Encuesta Nacional sobre Uso del Tiempo (ENUT) en el año 2019.

In [11]:
import folium
import pandas as pd
import panel as pn
import numpy as np
import plotly.graph_objs as go
pn.extension(sizing_mode = 'stretch_width')

In [17]:
df = pd.read_excel('2019UsoTiempo.xlsx')

In [39]:
## Variables
estados = {
    'Todos': 0,
    'Aguascalientes': 1,
    'Baja California': 2,
    'Baja California Sur': 3,
    'Campeche': 4,
    'Coahuila': 5,
    'Colima': 6,
    'Chiapas': 7,
    'Chihuahua': 8,
    'Ciudad de México': 9,
    'Durango': 10,
    'Guanajuato': 11,
    'Guerrero': 12,
    'Hidalgo': 13,
    'Jalisco': 14,
    'México': 15,
    'Michoacán': 16,
    'Morelos': 17,
    'Nayarit': 18,
    'Nuevo León': 19,
    'Oaxaca': 20,
    'Puebla': 21,
    'Querétaro': 22,
    'Quintana Roo': 23,
    'San Luis Potosí': 24,
    'Sinaloa': 25,
    'Sonora': 26,
    'Tabasco': 27,
    'Tamaulipas': 28,
    'Tlaxcala': 29,
    'Veracruz': 30,
    'Yucatán': 31,
    'Zacatecas': 32
}

In [14]:
# Cargar datos de ejemplo
data = {
    'Estado': ['Ciudad de México', 'Jalisco', 'Nuevo León', 'Veracruz', 'Puebla'],
    'Datos': [100, 200, 150, 180, 120]
}

df = pd.DataFrame(data)

# Coordenadas de los estados (pueden ser definidas manualmente)
coordenadas = {
    'Ciudad de México': (19.4326, -99.1332),
    'Jalisco': (20.6597, -103.3496),
    'Nuevo León': (25.5922, -99.9962),
    'Veracruz': (19.1738, -96.1342),
    'Puebla': (19.0414, -98.2063)
}

# Crear el mapa
m = folium.Map(location=[23.6345, -102.5528], zoom_start=5)

# Agregar marcadores para cada estado
for index, row in df.iterrows():
    estado = row['Estado']
    datos = row['Datos']
    coordenada = coordenadas[estado]
    folium.Marker(
        location=coordenada,
        popup=f"{estado}: {datos}",
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

In [37]:
# Widgets
is_man_checkbox = pn.widgets.Checkbox(name='Hombres', value=False)
is_woman_checkbox = pn.widgets.Checkbox(name='Mujeres', value=False)
age_range_slider = pn.widgets.IntRangeSlider(name='Rango de Edad', start=12, end=100, value=(20, 40))
is_working_select = pn.widgets.Select(name='¿Trabaja?', options=['Sí', 'No'])
is_studying_select = pn.widgets.Select(name='¿Estudia?', options=['Sí', 'No'])
state_select = pn.widgets.Select(name='Estado', options=[
    'Todos', 'Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Chiapas', 'Chihuahua',
    'Coahuila', 'Colima', 'Ciudad de México', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco',
    'Estado de México', 'Michoacán', 'Morelos', 'Nayarit', 'Nuevo León', 'Oaxaca', 'Puebla', 'Querétaro',
    'Quintana Roo', 'San Luis Potosí', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz',
    'Yucatán', 'Zacatecas'
])

In [43]:
# Función para actualizar el histograma
@pn.depends(is_man=is_man_checkbox.param.value, is_woman=is_woman_checkbox.param.value, age_range=age_range_slider.param.value, is_working=is_working_select.param.value, is_studying=is_studying_select.param.value, state=state_select.param.value)
def update_histogram(is_man, is_woman, age_range, is_working, is_studying, state):
    sexo = ""

    if (is_man and is_woman) or (is_man==False and is_woman==False):
      data = df[(df['SEXO'] == 1) | (df['SEXO'] == 2)]
      sexo = "Hombres y Mujeres"
    elif is_man:
      data = df[df['SEXO'] == 2]
      sexo = "Hombres"
    else:
      data = df[df['SEXO'] == 1]
      sexo = "Mujeres"

    if estados[state] > 0:
      data = data[data['ENT'] == estados[state]]
      sexo += " (" + state + ")"

    if is_working == "Sí":
      data = data[data['TRABAJA'] == 1]
    else:
      data = data[data['TRABAJA'] == 2]

    if is_studying == "Sí":
      data = data[data['ESCUELA'] == 1]
    else:
      data = data[data['ESCUELA'] == 2]

    data = data[(data['EDAD'] > age_range[0]) & (data['EDAD'] < age_range[1])]

    # Crear un gráfico de histograma interactivo con Plotly
    histogram = go.Figure(data=[go.Histogram(x=data['EDAD'])])

    histogram.update_layout(
        title=sexo,
        xaxis_title='Valor',
        yaxis_title='Frecuencia',
        bargap=0.05
    )

    return histogram


# Definir widgets para cada caja
widgets_box1 = pn.WidgetBox(
    pn.widgets.StaticText(value='Menu de opciones'),
    pn.widgets.StaticText(value='Selecciona el sexo de la población:'),
    is_man_checkbox,
    is_woman_checkbox,
    age_range_slider,
    is_working_select,
    is_studying_select,
    state_select,
    width=400,
    height=600
)

widgets_box2 = pn.pane.plot.Folium(m, width=600, height=600)

widgets_box3 = pn.pane.Plotly(update_histogram, width=600)

# Organizar cajas horizontales con espacio fijo entre ellas
layout = pn.Row(
    widgets_box1,
    pn.Spacer(width=25),  # Espacio fijo de 50 píxeles
    widgets_box2,
    pn.Spacer(width=25),  # Espacio fijo de 50 píxeles
    widgets_box3,
    sizing_mode='fixed'   # Establecer el modo de tamaño fijo
)

# Mostrar el layout
layout.servable()

14 
14 
14 
14 
14 
14 
14 
14 
14 
0 
0 
0 
0 
0 
0 
0 
14 
0 
20

Row(sizing_mode='fixed')
    [0] WidgetBox(height=600, sizing_mode='fixed', width=400)
        [0] StaticText(sizing_mode='stretch_width', value='Menu de opciones')
        [1] StaticText(sizing_mode='stretch_width', value='Selecciona el s...)
        [2] Checkbox(name='Hombres', sizing_mode='stretch_width', value=True)
        [3] Checkbox(name='Mujeres', sizing_mode='stretch_width')
        [4] IntRangeSlider(end=100, name='Rango de Edad', sizing_mode='stretch_width', start=12, value=(12, 73), value_end=73, value_start=12)
        [5] Select(name='¿Trabaja?', options=['Sí', 'No'], sizing_mode='stretch_width', value='Sí')
        [6] Select(name='¿Estudia?', options=['Sí', 'No'], sizing_mode='stretch_width', value='Sí')
        [7] Select(name='Estado', options=['Todos', 'Aguascalientes'...], sizing_mode='stretch_width', value='Jalisco')
    [1] Spacer(width=25)
    [2] Folium(Map, height=600, sizing_mode='fixed', width=600)
    [3] Spacer(width=25)
    [4] Plotly(Figure, width=600)